# Cleaning Melbourne sensor footfall data 
This script cleans the footfall data for Melbourne

#### Code initialisation

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from datetime import date, datetime

#### Import pedestrian count data

In [19]:
sensor_counts = pd.read_csv('../../Data/FootfallData/Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022.csv')
sensor_locations = pd.read_csv('../../Data/FootfallData/melbourne_locations.csv')

#### Rename columns to all lowercase (to facilitate joining)

In [20]:
sensor_counts.rename({'Date_Time': 'datetime', 'Year': 'year', 'Month':'month', 'Mdate': 'mdate', 
                      'Day': 'day', 'Time': 'time', 'Sensor_ID': 'sensor_id', 'Hourly_Counts': 'hourly_counts'}, 
                     axis = 1, inplace = True)

#### Drop unneeded columns

In [21]:
sensor_counts.drop(['ID', 'Sensor_Name'], axis = 1, inplace = True)
sensor_locations.drop(['sensor_description', 'sensor_name', 'installation_date', 'status', 'note', 'direction_1',
                      'direction_2'], axis = 1, inplace = True)

#### Join two dataframes so location and count info in same place

In [22]:
location_counts = pd.merge(sensor_locations, sensor_counts, on='sensor_id', how='inner')

#### It could be useful to drop sensors that do not have as many recorded countsfor now leaving them in, but may drop in the future

#### Properly format datetime column

In [23]:
location_counts['datetime'] = pd.to_datetime(location_counts['datetime'], format = '%B %d, %Y %I:%M:%S %p')

#### Order by datetime column

In [24]:
location_counts = location_counts.sort_values(by=['datetime'])
location_counts.reset_index(inplace = True, drop = True)

#### September, 2010 dates have problems:
All dates have a timestamp of 00:00, presume they are in order of hour of day, for each day there are only 23 hours worth of data
For now, in later stages just filter out 2010 data

In [25]:
location_counts = location_counts[location_counts['year']>2010]

### Clean and save data

In [28]:
# location_counts.drop(['Latitude', 'Longitude', 'location'], axis = 1, inplace =True)
# location_counts.rename({'day': 'weekday', 'mdate': 'day', 'time': 'hour'}, axis =1, inplace = True)

# Change month to integers 1-12
months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June':6, 'July': 7, 'August': 8,
         'September': 9, 'October': 10, 'November': 11, 'December': 12}

location_counts.month = location_counts.month.map(months)

In [30]:
just_some_sensors = location_counts[location_counts['sensor_id'].isin([2,6,8,9,10,11,18])]
just_some_sensors.to_csv("../../Cleaned_data/validsensors.csv",index=False)
location_counts.to_csv("../../Cleaned_data/allsensors.csv",index=False)

In [ ]:
# # Get data for just one sensor
# one_sensor =  location_counts[location_counts.sensor_id == 4]
# one_year = one_sensor[one_sensor.year==2010]
# # Set the datetime as the index
# one_sensor.set_index('datetime', inplace = True)
# duplicates = one_sensor[one_sensor.index.duplicated()]
# for day in duplicates.mdate.unique():
#     one_day = duplicates[duplicates.mdate == day]
#     print(day, len(one_day))